In [1]:
import glob
import tqdm
import nibabel as nib
import numpy as np
from nilearn.image import resample_img
import os

files = glob.glob(r'C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Cropped\*.nii.gz')
progress_bar = tqdm.tqdm(files, desc="Processing")

for file in progress_bar:
    nii_img = nib.load(file)
    voxel_size = nii_img.header.get_zooms()

    affine = nii_img.affine.copy()
    original_voxel_size = np.array(voxel_size)
    target_voxel_size = np.array([0.125, 0.125, 0.125])

    target_affine = affine.copy()
    target_affine[0, 0] = affine[0, 0] * (target_voxel_size[0] / original_voxel_size[0])
    target_affine[1, 1] = affine[1, 1] * (target_voxel_size[1] / original_voxel_size[1])
    target_affine[2, 2] = affine[2, 2] * (target_voxel_size[2] / original_voxel_size[2])

    resampled_img = resample_img(nii_img, target_affine=target_affine, interpolation='continuous', copy_header=True, force_resample=True)

    resampled_img.to_filename(rf'C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\{os.path.basename(file)}')

Processing: 100%|██████████| 147/147 [21:34<00:00,  8.81s/it]


In [2]:
def replace_zero_slices(nifti_path, output_path=None, threshold=0.90):
    """
    Check for slices that are mostly zeros in each plane (axial, sagittal, coronal).
    If a slice has more than threshold% zeros, replace all voxels in that slice with -2000.
    
    Parameters:
    -----------
    nifti_path : str
        Path to input .nii.gz file
    output_path : str, optional
        Path to save output file. If None, overwrites the input file.
    threshold : float, optional
        Percentage of zeros required to replace slice (default: 0.9 for 90%)
    """
    # Load the image
    img = nib.load(nifti_path)
    data = img.get_fdata().copy()  # Make a copy to modify
    
    print(f"Original image shape: {data.shape}")
    print(f"Threshold: {threshold*100}% zeros")
    
    # Track modifications
    modified_slices = {'axial': [], 'sagittal': [], 'coronal': []}
    
    # Check axial slices (along z-axis, axis=2)
    for z in range(data.shape[2]):
        slice_data = data[:, :, z]
        zero_percentage = np.sum(slice_data == 0) / slice_data.size
        if zero_percentage > threshold:
            data[:, :, z] = -2000
            modified_slices['axial'].append((z, f"{zero_percentage*100:.1f}%"))
    
    # Check sagittal slices (along x-axis, axis=0)
    for x in range(data.shape[0]):
        slice_data = data[x, :, :]
        zero_percentage = np.sum(slice_data == 0) / slice_data.size
        if zero_percentage > threshold:
            data[x, :, :] = -2000
            modified_slices['sagittal'].append((x, f"{zero_percentage*100:.1f}%"))
    
    # Check coronal slices (along y-axis, axis=1)
    for y in range(data.shape[1]):
        slice_data = data[:, y, :]
        zero_percentage = np.sum(slice_data == 0) / slice_data.size
        if zero_percentage > threshold:
            data[:, y, :] = -2000
            modified_slices['coronal'].append((y, f"{zero_percentage*100:.1f}%"))
    
    # Print summary
    print("\nModified slices:")
    print(f"  Axial (z-axis): {len(modified_slices['axial'])} slices")
    for idx, pct in modified_slices['axial']:
        print(f"    Slice {idx}: {pct} zeros")
    
    print(f"  Sagittal (x-axis): {len(modified_slices['sagittal'])} slices")
    for idx, pct in modified_slices['sagittal']:
        print(f"    Slice {idx}: {pct} zeros")
    
    print(f"  Coronal (y-axis): {len(modified_slices['coronal'])} slices")
    for idx, pct in modified_slices['coronal']:
        print(f"    Slice {idx}: {pct} zeros")
    
    # Create new image with modified data
    new_img = nib.Nifti1Image(data, img.affine, img.header)
    
    # Save the image
    if output_path is None:
        output_path = nifti_path
    
    nib.save(new_img, output_path)
    print(f"\nSaved modified image to: {output_path}")
    
    return modified_slices

files = glob.glob(r'C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\*.nii.gz')
progress_bar = tqdm.tqdm(files, desc="Processing")

for file in progress_bar:
    replace_zero_slices(file)

Processing:   0%|          | 0/147 [00:00<?, ?it/s]

Original image shape: (240, 257, 257)
Threshold: 90.0% zeros

Modified slices:
  Axial (z-axis): 1 slices
    Slice 256: 100.0% zeros
  Sagittal (x-axis): 1 slices
    Slice 239: 99.6% zeros
  Coronal (y-axis): 1 slices
    Slice 256: 99.2% zeros


Processing:   1%|          | 1/147 [00:01<02:28,  1.02s/it]


Saved modified image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\001_left_followup_mirrored_cropped.nii.gz
Original image shape: (254, 257, 257)
Threshold: 90.0% zeros

Modified slices:
  Axial (z-axis): 1 slices
    Slice 256: 100.0% zeros
  Sagittal (x-axis): 1 slices
    Slice 253: 99.6% zeros
  Coronal (y-axis): 1 slices
    Slice 256: 99.2% zeros


Processing:   1%|▏         | 2/147 [00:01<02:22,  1.01it/s]

...


Saved modified image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\107_right_followup_cropped.nii.gz
Original image shape: (257, 232, 257)
Threshold: 90.0% zeros

Modified slices:
  Axial (z-axis): 1 slices
    Slice 256: 100.0% zeros
  Sagittal (x-axis): 1 slices
    Slice 256: 99.6% zeros
  Coronal (y-axis): 1 slices
    Slice 231: 99.2% zeros


Processing:  99%|█████████▉| 146/147 [02:38<00:01,  1.09s/it]


Saved modified image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\110_left_followup_mirrored_cropped.nii.gz
Original image shape: (257, 257, 257)
Threshold: 90.0% zeros

Modified slices:
  Axial (z-axis): 1 slices
    Slice 256: 100.0% zeros
  Sagittal (x-axis): 1 slices
    Slice 256: 99.6% zeros
  Coronal (y-axis): 1 slices
    Slice 256: 99.2% zeros


Processing: 100%|██████████| 147/147 [02:39<00:00,  1.08s/it]


Saved modified image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Resized\110_right_followup_cropped.nii.gz


In [ ]:
def preprocess_volume(nifti_path, output_path=None, 
                      clip_min=-800, clip_max=2800,
                      target_shape=(256, 256, 256)):
    """
    Clip voxel intensities, rescale to [0, 1], and pad/crop to target shape.
    
    Parameters:
    -----------
    nifti_path : str
        Path to input .nii.gz file
    output_path : str, optional
        Path to save output file. If None, overwrites the input file.
    clip_min : float
        Minimum value for clipping (default: -800)
    clip_max : float
        Maximum value for clipping (default: 2800)
    target_shape : tuple
        Target shape for the volume (default: (256, 256, 256))
    """
    # Load the image
    img = nib.load(nifti_path)
    data = img.get_fdata().copy()
    
    print(f"Original shape: {data.shape}")
    print(f"Original intensity range: [{data.min():.2f}, {data.max():.2f}]")
    
    # Step 1: Clip intensities
    data = np.clip(data, clip_min, clip_max)
    # print(f"After clipping: [{data.min():.2f}, {data.max():.2f}]")
    data = -400 + ((data - clip_min)/(clip_max-clip_min))*(1600+400)
    data = np.round(data)
    # Step 2: Rescale to [0, 1]
    data = (data - clip_min) / (clip_max - clip_min)
    print(f"After rescaling: [{data.min():.4f}, {data.max():.4f}]")
    
    # Step 3: Pad or crop to target shape
    current_shape = np.array(data.shape)
    target_shape = np.array(target_shape)
    
    # Calculate padding or cropping needed for each dimension
    diff = target_shape - current_shape
    
    # Initialize the result array
    result = np.zeros(target_shape, dtype=data.dtype)
    
    # Calculate slicing indices for each dimension
    slices_src = []
    slices_dst = []
    
    for i in range(3):
        if diff[i] >= 0:  # Need to pad
            pad_before = diff[i] // 2
            slices_src.append(slice(0, current_shape[i]))
            slices_dst.append(slice(pad_before, pad_before + current_shape[i]))
        else:  # Need to crop
            crop_before = (-diff[i]) // 2
            slices_src.append(slice(crop_before, crop_before + target_shape[i]))
            slices_dst.append(slice(0, target_shape[i]))
    
    # Copy data with padding/cropping
    result[slices_dst[0], slices_dst[1], slices_dst[2]] = \
        data[slices_src[0], slices_src[1], slices_src[2]]
    
    print(f"Final shape: {result.shape}")
    
    # Update affine matrix to account for shape change
    new_affine = img.affine.copy()
    
    # Adjust origin for centering (if cropped or padded)
    for i in range(3):
        if diff[i] >= 0:  # Padded
            pad_before = diff[i] // 2
            new_affine[i, 3] -= pad_before * new_affine[i, i]
        else:  # Cropped
            crop_before = (-diff[i]) // 2
            new_affine[i, 3] += crop_before * new_affine[i, i]
    
    # Create new image
    new_img = nib.Nifti1Image(result, new_affine, img.header)
    
    # Save the image
    if output_path is None:
        output_path = nifti_path
    
    nib.save(new_img, output_path)
    print(f"\nSaved preprocessed image to: {output_path}")
    
    return result

files = glob.glob(r'C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done\*.nii.gz')
progress_bar = tqdm.tqdm(files, desc="Processing")

for file in progress_bar:
    preprocess_volume(file)

Processing:   0%|          | 0/147 [00:00<?, ?it/s]

Original shape: (240, 257, 257)
Original intensity range: [-2000.00, 2431.31]
After clipping: [-800.00, 2431.31]
After rescaling: [0.0000, 0.8976]
Final shape: (256, 256, 256)


Processing:   1%|          | 1/147 [00:01<02:58,  1.23s/it]


Saved preprocessed image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done\001_left_followup_mirrored_cropped.nii.gz
Original shape: (254, 257, 257)
Original intensity range: [-2000.00, 2241.00]
After clipping: [-800.00, 2241.00]
After rescaling: [0.0000, 0.8447]
Final shape: (256, 256, 256)


Processing:   1%|▏         | 2/147 [00:02<02:37,  1.08s/it]

...


Saved preprocessed image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done\107_right_followup_cropped.nii.gz
Original shape: (257, 232, 257)
Original intensity range: [-2000.00, 2597.00]
After clipping: [-800.00, 2597.00]
After rescaling: [0.0000, 0.9436]
Final shape: (256, 256, 256)


Processing:  99%|█████████▉| 146/147 [02:30<00:00,  1.03it/s]


Saved preprocessed image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done\110_left_followup_mirrored_cropped.nii.gz
Original shape: (257, 257, 257)
Original intensity range: [-2000.00, 2420.58]
After clipping: [-800.00, 2420.58]
After rescaling: [0.0000, 0.8946]
Final shape: (256, 256, 256)


Processing: 100%|██████████| 147/147 [02:31<00:00,  1.03s/it]


Saved preprocessed image to: C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done\110_right_followup_cropped.nii.gz


In [6]:
from pathlib import Path

# Rename all .nii.gz files
folder = Path(r'C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_preprocessed\Done')
for f in folder.glob('*.nii.gz'):
    new_name = f.name.replace('_mirrored','').replace('_mirror','').replace('_Mirror','').replace('_cropped','').replace('_baseline','').replace('_followup','').replace('_follow_up','').replace('_scan','')
    print(f"{f.name} > {new_name}")
    f.rename(f.parent / new_name)

001_left.nii.gz > 001_left.nii.gz
001_right.nii.gz > 001_right.nii.gz
002_left.nii.gz > 002_left.nii.gz
002_right.nii.gz > 002_right.nii.gz
003_left.nii.gz > 003_left.nii.gz
003_right.nii.gz > 003_right.nii.gz
004_left.nii.gz > 004_left.nii.gz
004_right.nii.gz > 004_right.nii.gz
005_left.nii.gz > 005_left.nii.gz
005_right.nii.gz > 005_right.nii.gz
006_left.nii.gz > 006_left.nii.gz
006_right.nii.gz > 006_right.nii.gz
010_left.nii.gz > 010_left.nii.gz
010_right.nii.gz > 010_right.nii.gz
012_left.nii.gz > 012_left.nii.gz
012_right.nii.gz > 012_right.nii.gz
013_left.nii.gz > 013_left.nii.gz
013_right.nii.gz > 013_right.nii.gz
014_left.nii.gz > 014_left.nii.gz
014_right.nii.gz > 014_right.nii.gz
015_left.nii.gz > 015_left.nii.gz
015_right.nii.gz > 015_right.nii.gz
017_left.nii.gz > 017_left.nii.gz
017_right.nii.gz > 017_right.nii.gz
019_left.nii.gz > 019_left.nii.gz
019_right.nii.gz > 019_right.nii.gz
021_left.nii.gz > 021_left.nii.gz
021_right.nii.gz > 021_right.nii.gz
023_left.nii.gz > 02